## Подготовка данных

In [ ]:
import pandas as pd
import numpy as np

data = pd.concat([pd.read_csv("titanic_train.csv"), pd.read_csv("titanic_test.csv")], ignore_index=True)

data["sex"] = pd.Categorical(data["sex"], categories=["male", "female"])
data["embarked"] = pd.Categorical(data["embarked"], categories=["C", "S", "Q"])
data["survived"] = pd.Categorical(data["survived"], categories=[0, 1])
# округлю возраст вверх, потому что странно иметь возраст 0,74, а еще графики так получаются красивее
data["age"] = data["age"].apply(np.ceil)

## Анализ таблицы

Выгрузим данные из таблиц, сконкатенируем test и train, а также категоризируем данные.

In [ ]:
val_counts = data["pclass"].value_counts()
print("Пассажиров класса 1", val_counts[1])
print("Пассажиров класса 2", val_counts[2])
print("Пассажиров класса 3", val_counts[3])

Пассажиров класса 1 323
Пассажиров класса 2 277
Пассажиров класса 3 709


Посмотрим сколько в каждом классе пассажиров. Как можно заметить, больше всего пассажиров в 3 классе.

In [143]:
grouped = data.groupby(["pclass", "sex"])["age"].mean()
print(grouped)

pclass  sex   
1       male      41.039735
        female    37.037594
2       male      30.829114
        female    27.504854
3       male      26.000000
        female    22.203947
Name: age, dtype: float64


/tmp/ipykernel_3325/2734801416.py:1: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



Как можно заметить самая молодая группа - это женщины 3 класса, а самая старая - мужчины первого.

In [146]:
print((data[(data["name"].str.startswith("K")) & (data["survived"]==1)]).sort_values(by = "fare", ascending=False))

     passenger_id  pclass                                        name     sex  \
171           180       1                      Kreuchen, Miss. Emilie  female   
778           176       1           Kenyon, Mrs. Frederick R (Marion)  female   
847           467       2         Kantor, Mrs. Sinai (Miriam Sternin)  female   
441           934       3         Kink-Heilmann, Miss. Luise Gretchen  female   
510           936       3  Kink-Heilmann, Mrs. Anton (Luise Heilmann)  female   
538           471       2               Kelly, Mrs. Florence "Fannie"  female   
578           916       3                          Karun, Miss. Manca  female   
626           469       2                         Keane, Miss. Nora A  female   
87            926       3                           Kennedy, Mr. John    male   
380           923       3                           Kelly, Miss. Mary  female   
734           922       3    Kelly, Miss. Anna Katherine "Annie Kate"  female   
800           941       3   

Больше всех заплатила Kreuchen, Miss. Emilie, меньше всех заплатил Krekorian, Mr. Neshan.

In [149]:
print("максимальное количество родных на корабле: ", max(data[data['survived'] == 1]["sibsp"] + data[data['survived'] == 1]["parch"]))

максимальное количество родных на корабле:  6


максимально на корабле находилось 6 родственников.

In [132]:
print("стоимость каюты у тех у кого указан номер в ", data[data["cabin"].notnull()]["fare"].mean() / data[data["cabin"].isnull()]["fare"].mean(), " раза больше, чем у тех, у кого не указан")

стоимость каюты у тех у кого указан номер в  4.282143526350036  раза больше, чем у тех, у кого не указан


стоимости отличаются больше, чем в 4 раза в пользу тех, у кого указан номер каюты.

## Визуализация

Гистограмма с тем сколько людей село в каждом месте посадки

In [133]:
import plotly.express as px
import plotly.graph_objects as go

px.histogram(data, x="embarked", nbins=20, title="сколько людей село в каждом из мест посадки")

заметно на графике скаттерплот ниже, что в основном те, у кого много братьев ехали с ними, предпочитали билеты подешевле.

In [134]:
px.scatter(data, x="sibsp", y="fare")

График на котором отобразим количество выцживших каждой возрастной группы, а также изначальное количество мужчин и женщин разных возрастов.

In [148]:
survived_counts = data[data['survived'] == 1].groupby('age').size().reset_index(name='count')
male_count = data[data["sex"] == "male"].groupby('age').size().reset_index(name='count')
female_count = data[data["sex"] == "female"].groupby('age').size().reset_index(name='count')

fig = go.Figure()

fig.add_trace(go.Scatter(x=survived_counts["age"], y=survived_counts["count"], mode="lines", name="отношение выживания к возрасту пассажира", line=dict(color='green')))
fig.add_trace(go.Scatter(x=male_count['age'], y=male_count["count"], mode='lines', name="количество мужчин определенного возраста", line=dict(color='red')))
fig.add_trace(go.Scatter(x=female_count['age'], y=female_count["count"], mode='lines', name="количество женщин определенного возраста", line=dict(color='blue')))


fig.show()

bar chart на котором отобразим соотношение полов только среди выживших всех возрастов.

In [136]:
survived_counts = data[data['survived'] == 1].groupby(['age', "sex"]).size().reset_index(name='count')
fig = px.bar(survived_counts, x="age", y="count", color="sex", title="соотношение полов среди выживших всех возрастов")
fig.show()

/tmp/ipykernel_3325/4037518609.py:1: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.

/usr/lib/python3/dist-packages/plotly/express/_core.py:1971: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



bar chart, где отобразим соотношение мест посадки среди выживших всех возрастов.

In [137]:
survived_embarked = data[data['survived'] == 1].groupby(['age', "embarked", "fare"]).size().reset_index(name='count')
fig = px.bar(survived_embarked, x="age", y="count", color="embarked", title="соотношение мест посадки среди выживших всех возрастов")
fig.show()

/tmp/ipykernel_3325/1218851053.py:1: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.

/usr/lib/python3/dist-packages/plotly/express/_core.py:1971: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



На графике ниже отобразим соотношение стоимости билета и количества выживших.

In [138]:
survived_fare = data[data['survived'] == 1].groupby(['fare']).size().reset_index(name='count')
fig = px.bar(survived_fare, x="fare", y="count", title="соотношение платы среди выживших", orientation="h")
fig.show()

Ниже приведен pie chart, где отображено соотношение полов на Титанике до крушения.

In [139]:
sex_counts = data['sex'].value_counts().reset_index()
sex_counts.columns = ['sex', 'count']

px.pie(sex_counts, values='count', names='sex', title="соотношение полов на корабле")

Ящик с усами, отображающий распределение возрастов на титанике, выделяя медиану, квартали и выбросы.

In [140]:
px.box(data, y="age")

sunburst отображающий не только соотношение полов, но и сколько людей какого пола где сели на корабль.

In [141]:
data_sun = data.dropna(subset=['sex', 'embarked']).reset_index(drop=True)

fig = px.sunburst(
    data_sun,
    path=['sex', 'embarked'],
    title="Разбиение по полу и месту посадки"
)

fig.show()

/usr/lib/python3/dist-packages/plotly/express/_core.py:1626: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.

/usr/lib/python3/dist-packages/plotly/express/_core.py:1626: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



трехмерный график, отображает выживших пассажиров распределяя их в трехмерном пространстве по 3 критерям: возраст, стоимость билета и класс.
Этот график помогает анализировать, как эти три фактора (возраст, класс и стоимость билета) могут быть связаны с вероятностью выживания пассажира.

In [ ]:
survived_3d = data[data['survived'] == 1].groupby(['age', 'pclass', 'fare']).size().reset_index(name='count')


fig = px.scatter_3d(
    survived_3d,
    x='age',
    y='pclass',
    z='fare',
    size='count',
    color='count',
    title='3D график выживших: по возрасту, стоимости билета и классу'
)
fig.show()